# Hidden Markov Model

Hidden Markov Model (HMM) is a statistical Markov model in which the system being modeled is assumed to be a Markov process with unobserved (i.e. hidden) states. The hidden Markov model can be represented as the simplest dynamic Bayesian network. The mathematics behind the HMM was developed by L. E. Baum and coworkers.

## Example

We have two states in Markov Model - Rainy and Sunny. We have two observations - Walk and Shop. We have the following probabilities:

- Initial probabilities: 
    - P(Rainy) = 0.6
    - P(Sunny) = 0.4
- Transition probabilities:
    - P(Rainy|Rainy) = 0.7
    - P(Sunny|Rainy) = 0.3
    - P(Sunny|Sunny) = 0.6
    - P(Rainy|Sunny) = 0.4
- Emission probabilities:
    - P(Walk|Rainy) = 0.1
    - P(Shop|Rainy) = 0.9
    - P(Walk|Sunny) = 0.6
    - P(Shop|Sunny) = 0.4

We have the following observations: Walk, Shop, Walk, Shop, Shop. We need to find the most probable sequence of states.

## Solution

We will use the Viterbi algorithm to solve this problem. The Viterbi algorithm is a dynamic programming algorithm for finding the most likely sequence of hidden states – called the Viterbi path – that results in a sequence of observed events, especially in the context of Markov information sources and hidden Markov models.    

The Viterbi algorithm is used to find the most likely sequence of hidden states. It is based on dynamic programming and the observation that the probability of the most likely path can be calculated recursively. The algorithm is used in many fields, including speech recognition, speech synthesis, part-of-speech tagging, gene prediction, and bioinformatics.

### Pseudocode



In [2]:
# so let's implement Hidden Markov Model

import numpy as np
import pandas as pd
# prints python and library versions
import sys
print('Python: {}'.format(sys.version))
print('Numpy: {}'.format(np.__version__))
print('Pandas: {}'.format(pd.__version__))

Python: 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)]
Numpy: 1.23.5
Pandas: 1.5.2


In [5]:
# now let's define the model
class HiddenMarkovModel:
    def __init__(self, A, B, pi):
        self.A = A
        self.B = B
        self.pi = pi
        self.N = A.shape[0]
        self.M = B.shape[1]
        
    def forward(self, O):
        T = len(O)
        alpha = np.zeros((T, self.N))
        alpha[0] = self.pi * self.B[:, O[0]]
        for t in range(1, T):
            alpha[t] = self.B[:, O[t]] * np.dot(alpha[t-1], self.A)
        return alpha
    
    def backward(self, O):
        T = len(O)
        beta = np.zeros((T, self.N))
        beta[T-1] = 1
        for t in range(T-2, -1, -1):
            beta[t] = np.dot(self.A, self.B[:, O[t+1]] * beta[t+1])
        return beta
    
    def fit(self, O):
        T = len(O)
        alpha = self.forward(O)
        beta = self.backward(O)
        xi = np.zeros((T-1, self.N, self.N))
        for t in range(T-1):
            xi[t] = (alpha[t].reshape(-1, 1) * self.A * self.B[:, O[t+1]].reshape(1, -1) * beta[t+1].reshape(1, -1)) / np.dot(alpha[t], np.dot(self.A, self.B[:, O[t+1]] * beta[t+1]))
        gamma = alpha * beta / np.sum(alpha * beta, axis=1).reshape(-1, 1)
        return xi, gamma
    
    def viterbi(self, O):
        T = len(O)
        delta = np.zeros((T, self.N))
        psi = np.zeros((T, self.N))
        delta[0] = self.pi * self.B[:, O[0]]
        for t in range(1, T):
            for j in range(self.N):
                delta[t, j] = np.max(delta[t-1] * self.A[:, j]) * self.B[j, O[t]]
                psi[t, j] = np.argmax(delta[t-1] * self.A[:, j])
        q = np.zeros(T, dtype=np.int32)
        q[T-1] = np.argmax(delta[T-1])
        return q

# let's define the model parameters
A = np.array([[0.7, 0.3], [0.4, 0.6]]) # our transition matrix for the hidden states
B = np.array([[0.1, 0.4, 0.5], [0.7, 0.2, 0.1]]) # our emission matrix for the observations
pi = np.array([0.6, 0.4]) # initial state distribution

# let's define the observations
O = np.array([0, 1, 2, 2, 1, 0]) # so imagine them being 0=red, 1=green, 2=blue
# let's create the model
hmm = HiddenMarkovModel(A, B, pi)
# let's do the forward algorithm
alpha = hmm.forward(O)
print(alpha)
# let's do the backward algorithm
beta = hmm.backward(O)
print(beta)
# let's do the viterbi algorithm
q = hmm.viterbi(O)
print(q)


[[6.0000000e-02 2.8000000e-01]
 [6.1600000e-02 3.7200000e-02]
 [2.9000000e-02 4.0800000e-03]
 [1.0966000e-02 1.1148000e-03]
 [3.2488480e-03 7.9173600e-04]
 [2.5908880e-04 1.0147872e-03]]
[[0.0047374 0.0035344]
 [0.014851  0.009652 ]
 [0.0401    0.0272   ]
 [0.106     0.1      ]
 [0.28      0.46     ]
 [1.        1.       ]]
[0 0 0 0 0 1]


In [ ]:
# so it is saying that the most likely sequence of hidden states is 0, 0, 0, 0, 0, 1

# TODO add the Baum-Welch algorithm for training the model
# TODO add mapping of numerical values to the actual states and observations